# Working with IP addresses

Python provides a number of helpful utilities for
dealing with IP addresses in the standard library module `ipaddress`. There's a
number of 3rd party tools that can also help with these tasks.

## Validating IP Addresses

In [12]:
import ipaddress

In [18]:
try:
    addr = ipaddress.IPv4Address("3000.200.100.0")
except ipaddress.AddressValueError:
    print("Invalid address")
else:
    print("Valid address")

Invalid address


## Finding your local IP address

There are a few ways you can find your local IP address.

\*nix

In [ ]:
ifconfig

Windows

In [ ]:
ipconfig

Typical output:

In [ ]:
%%bash
$ ifconfig
br-0fc40b53eac2 Link encap:Ethernet  HWaddr
02:42:9a:57:42:94  
          inet addr:172.18.0.1  Bcast:172.18.255.255
Mask:255.255.0.0
          UP BROADCAST MULTICAST  MTU:1500  Metric:1
RX packets:0 errors:0 dropped:0 overruns:0 frame:0
          TX packets:0
errors:0 dropped:0 overruns:0 carrier:0
          collisions:0 txqueuelen:0
RX bytes:0 (0.0 B)  TX bytes:0 (0.0 B)

docker0   Link encap:Ethernet  HWaddr
02:42:f2:be:5a:21  
          inet addr:172.17.0.1  Bcast:172.17.255.255
Mask:255.255.0.0
          UP BROADCAST MULTICAST  MTU:1500  Metric:1
RX packets:0 errors:0 dropped:0 overruns:0 frame:0
          TX packets:0
errors:0 dropped:0 overruns:0 carrier:0
          collisions:0 txqueuelen:0
RX bytes:0 (0.0 B)  TX bytes:0 (0.0 B)

enp5s0    Link encap:Ethernet  HWaddr
d8:cb:8a:c2:29:4b  
          inet addr:192.168.1.13  Bcast:192.168.1.255
Mask:255.255.255.0
          inet6 addr: fe80::8610:baf9:b53e:2cdf/64 Scope:Link
UP BROADCAST RUNNING MULTICAST  MTU:1500  Metric:1
          RX packets:10476727
errors:0 dropped:0 overruns:0 frame:0
          TX packets:9224863 errors:0
dropped:0 overruns:0 carrier:0
          collisions:0 txqueuelen:1000
RX bytes:12246353993 (12.2 GB)  TX bytes:5848871311 (5.8 GB)
Memory:fb200000-fb2fffff 

lo        Link encap:Local Loopback  
          inet
addr:127.0.0.1  Mask:255.0.0.0
          inet6 addr: ::1/128 Scope:Host
UP LOOPBACK RUNNING  MTU:65536  Metric:1
          RX packets:170011 errors:0
dropped:0 overruns:0 frame:0
          TX packets:170011 errors:0 dropped:0
overruns:0 carrier:0
          collisions:0 txqueuelen:1 
          RX
bytes:127632763 (127.6 MB)  TX bytes:127632763 (127.6 MB)

## Finding adaptor info from Python

You could run a system command and parse the
text to find information about IP addresses, but there's a better way:

In [ ]:
pip install netifaces

This library will allow you to find adaptor information without
having to parse text.

In [3]:
import netifaces

In [4]:
interfaces = netifaces.interfaces()

In [10]:
interfaces

['lo', 'enp5s0', 'br-0fc40b53eac2', 'docker0']

In [9]:
for iface in interfaces:
    ipaddrs = netifaces.ifaddresses(iface)
    if netifaces.AF_INET in ipaddrs:
        ipaddr_desc = ipaddrs[netifaces.AF_INET][0]
        print(f"Network interface: {iface}")
        if 'addr' in ipaddr_desc:
            print(f"\tIP address: {ipaddr_desc['addr']}")
        if 'netmask' in ipaddr_desc:
            print(f"\tNetmask: {ipaddr_desc['netmask']}")

Network interface: lo
	IP address: 127.0.0.1
	Netmask: 255.0.0.0
Network interface: enp5s0
	IP address: 192.168.1.13
	Netmask: 255.255.255.0
Network interface: br-0fc40b53eac2
	IP address: 172.18.0.1
	Netmask: 255.255.0.0
Network interface: docker0
	IP address: 172.17.0.1
	Netmask: 255.255.0.0


## Finding properties of interfaces

The ipaddress module gives you some
convenience functions for dealing with interfaces:

In [47]:
import ipaddress

In [48]:
loopback = ipaddress.IPv4Interface("127.0.0.1")

In [49]:
loopback.is_loopback

True

In [50]:
loopback.is_multicast

False

In [51]:
loopback.is_private

True

## IP networks

The `ipaddress` module provides helpers for IP networks too

In [20]:
example = ipaddress.ip_network("192.168.0.0/16")

In [21]:
example.netmask

IPv4Address('255.255.0.0')

In [22]:
example.broadcast_address

IPv4Address('192.168.255.255')

In [23]:
example.network_address

IPv4Address('192.168.0.0')

In [25]:
addresses_in_subnet = example.num_addresses
addresses_in_subnet

65536

## Finding subnet address blocks

Consider the network "192.168.5.0/28". We could figure out which addresses are
on this subnet by hand, but we can also use a utility `ipaddress.ip_network`.
This provides a generator which will yield out the network addresses on that
subnet

In [27]:
net = ipaddress.ip_network("192.168.5.0/28")

## Exercise: IP addresses on a subnet

Write a function that will take an IP address and a subnet mask length and
return a list of the addresses on that subnet.

- Hint: the `ipaddress.IPv4Interface` may be convenient.

In [44]:
# See solutions/subnet_addresses.py

Once you have written a function `get_subnet_addresses()`, you should be able
to use it like this:

In [45]:
address = ipaddress.IPv4Address("192.168.1.1")
subnet_addresses = get_subnet_addresses(address, 28)
subnet_addresses

[IPv4Address('192.168.1.1'),
 IPv4Address('192.168.1.2'),
 IPv4Address('192.168.1.3'),
 IPv4Address('192.168.1.4'),
 IPv4Address('192.168.1.5'),
 IPv4Address('192.168.1.6'),
 IPv4Address('192.168.1.7'),
 IPv4Address('192.168.1.8'),
 IPv4Address('192.168.1.9'),
 IPv4Address('192.168.1.10'),
 IPv4Address('192.168.1.11'),
 IPv4Address('192.168.1.12'),
 IPv4Address('192.168.1.13'),
 IPv4Address('192.168.1.14')]